# Reading note for "RandLA-Net"

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch_points_kernels import knn

In [4]:
rand_idx = np.random.choice(100, 10)
print(f"rand_idx: {rand_idx}")

rand_idx: [69 15  5 77  6 97 25 89 33 12]


In [5]:
# fake data
torch.manual_seed(0)

BATCH = 8
NUM_POINT = 2**10
D_XYZ = 3
D_IN = 4
NUM_NEIGHBOUR = 16

pc = torch.rand(BATCH, NUM_POINT, D_XYZ + D_IN)
print(f"pc: {pc.shape}")
pc_xyz = pc[:, :, :3]
print(f"pc_xyz: {pc_xyz.shape}")
pc_feat = pc[:, :, 3:]
print(f"pc_feat: {pc_feat.shape}")

pc: torch.Size([8, 1024, 7])
pc_xyz: torch.Size([8, 1024, 3])
pc_feat: torch.Size([8, 1024, 4])


In [6]:
permutation = torch.randperm(100)
print(f"permutation min: {permutation.min()}\npermutation max: {permutation.max()}")
xyz_perm = pc_xyz[:,permutation]
print(f"xyz_perm: {xyz_perm.shape}")

permutation min: 0
permutation max: 99
xyz_perm: torch.Size([8, 100, 3])


In [7]:
idx, dist = knn(pc_xyz[:, :100].cpu().contiguous(), pc_xyz[:, :1000].cpu().contiguous(), 1)
print(f"idx: {idx.shape}")

idx: torch.Size([8, 1000, 1])


In [8]:
# extended_idx = idx.unsqueeze(1).repeat(1, 3, 1, 1)
# print(f"extended_idx: {extended_idx.shape}")
# extended_dist = dist.unsqueeze(1).repeat(1, 3, 1, 1)
# print(f"extended_dist: {extended_dist.shape}")
# extended_xyz = pc_xyz.transpose(-2, -1).unsqueeze(-1).repeat(1, 1, 1, NUM_NEIGHBOUR)
# print(f"extended_xyz: {extended_xyz.shape}")
# neighbour = extended_xyz.gather(dim=2, index=extended_idx)
# print(f"neighbour: {neighbour.shape}")

In [9]:
class MLP(nn.Module):
    def __init__(self, d_in, d_out, kernel_size=1, stride=1, bn=False, activation_fn=None):
        super(MLP, self).__init__()

        self.conv = nn.Conv2d(in_channels=d_in, out_channels=d_out, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(d_out) if bn else None
        self.activation_fn = activation_fn

    def forward(self, x):
        """
        Input:
            x: [B, d_in, N, K]
        Output:
            x: [B, d_out, N, K]

        """

        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.activation_fn is not None:
            x = self.activation_fn(x)

        return x

In [10]:
class LocalSpatialEncoding(nn.Module):
    def __init__(self, d_out):
        super(LocalSpatialEncoding, self).__init__()

        self.mlp = MLP(d_in=3+3+3+1, d_out=d_out, bn=True, activation_fn=nn.ReLU())

    def forward(self, feat, xyz, knn_output):
        '''
        Input:
            feat: [B, d_in, N, 1]
            xyz: [B, N, 3]
            knn_output: [B, N, K]
        Output:
            neighbouring_feat: [B, 2*d_out, N, K]

        '''

        idx, dist = knn_output  # [B, N, K]
        B, N, K = idx.size()

        extended_idx = idx.unsqueeze(1).repeat(1, 3, 1, 1)  # [B, 3, N, K]
        extended_xyz = xyz.transpose(-2, -1).unsqueeze(-1).repeat(1, 1, 1, K)  # [B, 3, N, K]
        neighbour = extended_xyz.gather(dim=2, index=extended_idx)  # [B, 3, N, K]
        concat_xyz = torch.cat((extended_xyz, neighbour, extended_xyz - neighbour, dist.unsqueeze(1)), dim=1) # [B, 10, N, K]
        relative_pnt_pos_enc = self.mlp(concat_xyz)  # [B, d_out, N, K]
        output = torch.cat((relative_pnt_pos_enc, feat.repeat(1, 1, 1, K)), dim=1)  # [B, 2*d_out, N, K]

        return output

In [11]:
LoSE = LocalSpatialEncoding(d_out=4)
knn_output = knn(pc_xyz.cpu().contiguous(), pc_xyz.cpu().contiguous(), NUM_NEIGHBOUR)
lose_feat = LoSE(pc_feat.transpose(-2,-1).unsqueeze(-1), pc_xyz, knn_output)
print(f"LoSE features: {lose_feat.shape}")

LoSE features: torch.Size([8, 8, 1024, 16])


In [12]:
class AttentivePooling(nn.Module):
    def __init__(self, d_in, d_out):
        super(AttentivePooling, self).__init__()

        self.score_fn = nn.Sequential(
            nn.Linear(d_in, d_in),
            nn.Softmax(dim=-2)
        )

        self.mlp = MLP(d_in=d_in, d_out=d_out)

    def forward(self, feat):
        '''
        Input:
            feat: [B, d_in, N, K]
        Output:
            agg_feat: [B, d_out, N, 1]
        '''
        scores = self.score_fn(feat.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)  # [B, N, K, d_in] -> [B, d_in, N, K]
        feat = torch.sum(scores*feat, dim=-1, keepdim=True)  # [B, d_in, N, 1]
        agg_feat = self.mlp(feat)  # [B, d_out, N, 1]

        return agg_feat

In [13]:
AttPooling = AttentivePooling(d_in=2*D_IN, d_out=32)
agg_feat = AttPooling(lose_feat)
print(f"aggregated features: {agg_feat.shape}")

aggregated features: torch.Size([8, 32, 1024, 1])


In [14]:
class LocalFeatureAggregation(nn.Module):
    def __init__(self, d_in, d_out, num_neighbours):
        super(LocalFeatureAggregation, self).__init__()

        self.num_neighbours = num_neighbours

        self.mlp1 = MLP(d_in=d_in, d_out=d_out//2)
        self.mlp2 = MLP(d_in=d_out, d_out=2*d_out)
        self.mlp3 = MLP(d_in=d_in, d_out=2*d_out, bn=True)

        self.lose1 = LocalSpatialEncoding(d_out=d_out//2)
        self.lose2 = LocalSpatialEncoding(d_out=d_out//2)

        self.pool1 = AttentivePooling(d_in=d_out, d_out=d_out//2)
        self.pool2 = AttentivePooling(d_in=d_out, d_out=d_out)

        self.lrelu = nn.LeakyReLU()

    def forward(self, feat, xyz):
        '''
        Input:
            feat: [B, d_in, N, 1]
            xyz: [B, N, 3]
        Output:
            aggregated_feat: [B, N, 2*d_out]
        '''
        knn_output = knn(xyz.cpu().contiguous(), xyz.cpu().contiguous(), self.num_neighbours)  # [B, N, K]

        residual = self.mlp3(feat)  # [B, 2*d_out, N, 1]

        feat1 = self.mlp1(feat)  # [B, d_out//2, N, 1]
        lose_feat1 = self.lose1(feat1, xyz, knn_output)  # [B, d_out, N, K]
        att_feat1 = self.pool1(lose_feat1)  # [B, d_out//2, N, 1]

        lose_feat2 = self.lose2(att_feat1, xyz, knn_output)  # [B, d_out, N, K]
        att_feat2 = self.pool2(lose_feat2)  # [B, d_out, N, 1]
        
        feat2 = self.mlp2(att_feat2)  # [B, 2*d_out, N, 1]

        return self.lrelu(feat2 + residual)

In [15]:
lfa = LocalFeatureAggregation(d_in=D_IN, d_out=16, num_neighbours=NUM_NEIGHBOUR)
aggregated_feat = lfa(pc_feat.transpose(-2,-1).unsqueeze(-1), pc_xyz)
print(f"Aggregated feat: {aggregated_feat.shape}")

Aggregated feat: torch.Size([8, 32, 1024, 1])


In [16]:
class RandLANet(nn.Module):
    def __init__(self, d_in, num_classes, num_neighbours=16, decimation=4, dropout=0.5):
        super(RandLANet, self).__init__()

        self.num_neighbours = num_neighbours
        self.decimation = decimation
        self.fc1 = nn.Linear(in_features=d_in, out_features=8)

        self.encoder = nn.ModuleList([
            LocalFeatureAggregation(d_in=8, d_out=16, num_neighbours=num_neighbours),
            LocalFeatureAggregation(2*16, 64, num_neighbours),
            LocalFeatureAggregation(2*64, 128, num_neighbours),
            LocalFeatureAggregation(2*128, 256, num_neighbours)
        ])

        self.mlp = MLP(d_in=512, d_out=512, activation_fn=nn.ReLU())

        self.decoder = nn.ModuleList([
            MLP(d_in=512, d_out=256, bn=True, activation_fn=nn.ReLU()),
            MLP(d_in=2*256, d_out=128, bn=True, activation_fn=nn.ReLU()),
            MLP(d_in=2*128, d_out=32, bn=True, activation_fn=nn.ReLU()),
            MLP(d_in=2*32, d_out=8, bn=True, activation_fn=nn.ReLU())
        ])

        self.fc_final = nn.Sequential(
            MLP(d_in=8+8, d_out=64, bn=True, activation_fn=nn.ReLU()),
            MLP(d_in=64, d_out=32, bn=True, activation_fn=nn.ReLU()),
            nn.Dropout(p=dropout),
            MLP(d_in=32, d_out=num_classes)
        )

    def forward(self, points):
        '''
        Input:
            points: [B, N, 3+d_in]
        Output:
            class scores: [B, num_classes, N]
        '''

        B, N, _ = points.size()
        d = self.decimation
        decimation_ratio = 1

        xyz = points[..., :3]  # [B, N, 3]

        feat_stack = []

        feat = self.fc1(points).transpose(-2, -1).unsqueeze(-1)  # [B, 8, N, 1]

        for lfa in self.encoder:
            feat_stack.append(feat)
            decimation_ratio *= d
            feat = lfa(feat[:, :, :N//decimation_ratio], xyz[:, :N//decimation_ratio])  # [B, 2*d_out, N//decimation_ratio, 1]
        # [B, 512, N//256, 1]

        feat = self.mlp(feat)  # [B, 512, N//256, 1]

        for mlp in self.decoder:
            feat = mlp(feat)
            # find one nearset neighbour for each upsampled point in the downsampled set
            idx, _ = knn(xyz[:, :N//decimation_ratio].contiguous(), xyz[:,:d*N//decimation_ratio].contiguous(), 1)  # [B, d*N//decimation, 1]
            extended_idx = idx.unsqueeze(1).repeat(1, feat.size(1), 1, 1)  # [B, d_out, d*N//decimation, 1]
            print(f"extended_idx: {extended_idx.shape}")
            feat_neighbour = torch.gather(feat, -2, extended_idx)  # [B, d_out, d*N//decimation, 1]
            print(f"feat_neighbour: {feat_neighbour.shape}")
            feat_pop = feat_stack.pop()  # [B, d_out, d*N//decimation, 1]
            print(f"feat_pop: {feat_pop.shape}")
            feat = torch.cat((feat_neighbour, feat_pop), dim=1)
            # feat = mlp(feat)
            decimation_ratio //= d
        
        scores = self.fc_final(feat)  # [B, num_classes, N, 1]

        return scores.squeeze(-1)

In [24]:
model = RandLANet(d_in=D_XYZ+D_IN, num_classes=40, num_neighbours=4)
class_scores = model(pc)
print(f"class scores: {class_scores.shape}")
class_label = class_scores.transpose(-2, -1).max(-1)
print(f"class label: {class_label[0].shape}")

extended_idx: torch.Size([8, 256, 16, 1])
feat_neighbour: torch.Size([8, 256, 16, 1])
feat_pop: torch.Size([8, 256, 16, 1])
extended_idx: torch.Size([8, 128, 64, 1])
feat_neighbour: torch.Size([8, 128, 64, 1])
feat_pop: torch.Size([8, 128, 64, 1])
extended_idx: torch.Size([8, 32, 256, 1])
feat_neighbour: torch.Size([8, 32, 256, 1])
feat_pop: torch.Size([8, 32, 256, 1])
extended_idx: torch.Size([8, 8, 1024, 1])
feat_neighbour: torch.Size([8, 8, 1024, 1])
feat_pop: torch.Size([8, 8, 1024, 1])
class scores: torch.Size([8, 40, 1024])
class label: torch.Size([8, 1024])
